<a href="https://colab.research.google.com/github/JanEggers-hr/youtube-scraper/blob/main/youtube_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Youtube-Scraper v02

Holt die Videos des übergebenen Youtube-Kanals, isoliert das Audio, und verschriftlicht sie mit einem Speech-to-Text-Service. 

Die einzelnen Schritte:
- Mit youtube_dl nacheinander Videos holen, zu MP3 wandeln
- Die dabei gesammelten wichtigsten Metadaten (Upload-Datum, View-Count) in einer Excel-Datei ablegen
- Mit der OpenAI-Library "Whisper" in Text konvertieren

Dateien liegen danach alle in einem Ordner "output" und müssen von da exportiert werden, weil sie sonst am Ende der Colab-Laufzeit gelöscht werden. Wär doch schade drum. 

## N00bwarnung

Das Skript ist nicht sehr elegant, vor allem aber ist es desaströs langsam. Die youtube_dl-Library lässt sich, soweit ich das erforschen konnte, nicht asynchron ausführen, was schade ist - man könnte ja durchaus mehrere Videos gleichzeitig herunterladen. Vor allem aber könnte man sie Whisper schon mal zum Transkribieren geben. 

So immerhin ist alles schön einfach - wenn auch nicht schnell. 

## GPU einschalten!

Die Whisper-Library profitiert sehr davon, wenn man im Menü unter "Laufzeit/Laufzeittyp ändern" die GPU aktiviert. (Auch wenn Google zunächst meckert, weil das Herunterladen der Videos ohne GPU-Nutzung abgeht.)

Theoretisch könnte man vermutlich auch eine ffmpeg-Variante einbinden, die die GPU nutzt, dann geht der YT-Download schneller... aber: siehe oben. 

## ----
### Changelog
* v02 - Fehler beim Download automatisch auffangen (ganz simpel: Download nochmal starten)
* v01 - Suche nach noch nicht heruntergeladenen Videos; Vervollständigung
* v00 - Funktioniert

### Todo: Mögliche Verbesserungen

- Publikationsdatum der Videos in den Dateinamen, um besseren Überblick zu haben


In [ ]:
# Vorbereitung: youtube_dl installieren
!pip install youtube_dl

In [ ]:
# Für die Datensicherung: Drive verbinden
import os
from google.colab import drive
drive.mount('/content/gdrive')

# Ausgabeverzeichnis (/gdrive/MyDrive/youtube-scraper/output) anlegen: 
if not os.path.exists("/content/gdrive/MyDrive/youtube-scraper"):
    os.mkdir("/content/gdrive/MyDrive/youtube-scraper")
if not os.path.exists("/content/gdrive/MyDrive/youtube-scraper/output"):
    os.mkdir("/content/gdrive/MyDrive/youtube-scraper/output")

path = "/content/gdrive/MyDrive/youtube-scraper/output"
os.chdir(path)

Hier den Kanal eintragen, der gescraped werden soll. 

In [ ]:
channel_url = "https://www.youtube.com/@JanEggers"

Mit der Library youtube_dl wird eine Liste der Videos mit Metadaten als Tabelle erstellt. Das wird vom Download getrennt, um Abbrüche auffangen zu können - manchmal scheitert youtube_dl an einem "403 FORBIDDEN" der Plattform. 

Also: in der ersten Runde nur Daten sammeln - und als XLSX exportieren. 

In [ ]:
from __future__ import unicode_literals
import youtube_dl
import pandas as pd

# Die Optionen, um neben den Metadaten gleich alle MP3-Dateien herunterzuladen: 
ydl_opts = { 'quiet': 'True' }

# Erste Aufgabe: Hole Metadaten für einen Channel, keine Downloads
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    metadata = ydl.extract_info(channel_url, download=False) 

# Die Daten sind ein bisserl verschachtelt: Die URLs der Videos sind als Dictionary in einer Liste von Dictionaries oder so. Das hier funktioniert
videos_df = pd.DataFrame(metadata['entries'][0]['entries'], columns=["id","upload_date","description","duration","view_count","average_rating",
                                                                     "age_limit","categories","tags"])

# Liste exportieren
videos_df.to_excel("video_liste.xlsx")
print(len(videos_df)," Videos in der Playlist/Kanal-Startseite gefunden.")
videos_df.head(5)

## Videos herunterladen

*ACHTUNG*: Dieser Schritt dauert eine Weile - und bricht gern mal mit einem Fehler ab, weil Youtube dem youtube-dl-Skript gerne mal ein "Darfste nicht!" in den Weg wirft. **Falls youtube-dl mit einem Fehler abbricht, ruft die Funktion sich selbst noch mal neu auf.** Das stößt (zum Glück) irgendwann an Grenzen - bei zu vielen Rekursionen bricht Python ab. 

Dummerweise verliert das Colab-Notebook nach einer Zeit die virtuelle Maschine, und man muss alles nochmal von vorn starten - deshalb schaut der Code, welche Videos schon heruntergeladen sind, und macht da weiter, wo es zuletzt aufgehört hat. 

In [ ]:
# Definiere den Download als Funktion - die sich im Fehlerfall rekursiv neu aufruft

def download_mp3(videos_liste):
    # Die Optionen, um neben den Metadaten gleich alle MP3-Dateien herunterzuladen: 
    ydl_opts = {'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '128',
    }],
    'outtmpl': '%(id)s.%(ext)s', # Formatiere Dateinamen: id.mp3
    }

    # Leere Liste anlegen
    new_urls = []
    # Videos, für die es noch kein mp3 gibt, in die Liste
    for id in videos_liste:
        f = path + "/" + id + ".mp3"
        if not os.path.exists(f):
            new_urls.append(id)

    if len(new_urls) > 0:
        print("Noch ",len(new_urls)," Videos herunterladen...")
        # Die Liste an den Downloader verfüttern.
        # Videos werden nach dem Download in MP3 gewandelt, das
        # bestimmen die Parameter. 
        try: 
            with youtube_dl.YoutubeDL(ydl_opts) as ydl:
                    ydl.download(new_urls)
        except:
            # Fehler geworfen; versuch es nochmal
            print("Versuche es nochmal...")
            download_mp3(videos_liste)

    print("Alle Videos des Kanals heruntergeladen!")
    return(True)

# Jetzt die Funktion ausführen
download_mp3(videos_df["id"])

Die Audios liegen alle als MP3 im Ordner output - mit den Dateinamen (id).mp3. Jetzt alle an den STT-Konverter schicken. 

Wir versuchen hier an dieser Stelle mal, OpenAIs "Whisper" einzusetzen. 

Quelle: https://github.com/openai/whisper

In [ ]:
!pip install git+https://github.com/openai/whisper.git 

Wenn die Installation der Library von Github geklappt hat, ist die eigentliche Transkription ziemlich simpel: 

In [ ]:
import whisper
model = whisper.load_model("medium")

# Wie oben: Liste von allen
new_urls = []
for id in videos_df["id"]:
    f = path + "/" + id + "_transcribe.txt"
    if not os.path.exists(f):
        new_urls.append(id)

print(len(new_urls)," MP3-Dateien zu verschriftlichen.")

for id in new_urls:
    mp3_fname = path + "/" + id + ".mp3"
    txt_fname = path + "/" + id + "_transcribe.txt"

    result = model.transcribe(mp3_fname)
    # Ergebnis der Umwandlung als Textdatei ausgeben
    with open(txt_fname, 'w') as f:
      f.write(result["text"])
    print(txt_fname," erzeugt")

print("Fertig - ",len(new_urls)," Dateien konvertiert.")